In [1]:
import pandas as pd

In [2]:
stock=pd.read_csv('retdata.csv',sep='\t')
stock.head(n=3)

Date  Hon Hai Precision  TSMC           Uni-President
0  2014-01-02             0.3745        -0.9479         0.5587
1  2014-01-03            -1.2438        -1.9139        -0.9259
2  2014-01-06            -0.5038         0.0000         0.0000

In [4]:
stock.index = pd.to_datetime(stock.Date)
UniPresident = stock.iloc[:,3]
UniPresident.head(n=3)

Date
2014-01-02    0.5587
2014-01-03   -0.9259
2014-01-06    0.0000
Name: Uni-President, dtype: float64

In [9]:
#讀入三因子資料
factors=pd.read_csv('factors.csv')
factors.head(n=3)

Security Name                         CoName    YY/MM/DD  \
0       Z8888    TSEC+GreTai Non-Banking index  2014/01/02   
1       Z8888    TSEC+GreTai Non-Banking index  2014/01/03   
2       Z8888    TSEC+GreTai Non-Banking index  2014/01/06   

   Market Risk Premium  Size Factor (3 Factor)  Book to Market Factor  \
0               0.0145                  0.7197                 0.3039   
1              -0.5858                  0.6868                 1.0533   
2              -0.4534                  0.5783                 0.4498   

   Risk-free Interest Rate  
0                    1.355  
1                    1.355  
2                    1.355

In [33]:
factors.index =pd.to_datetime(factors['YY/MM/DD']) 
factors.columns

Index(['Security Name', 'CoName', 'YY/MM/DD', 'Market Risk Premium',
       'Size Factor (3 Factor)', 'Book to Market Factor',
       'Risk-free Interest Rate'],
      dtype='object')

In [36]:
#將年無風險利率轉化成日無風險利率
factors['Risk-free Interest Rate']=(factors['Risk-free Interest Rate']**(1/360)-1)*100
factors.head(n=3)

Security Name                         CoName    YY/MM/DD  \
YY/MM/DD                                                              
2014-01-02       Z8888    TSEC+GreTai Non-Banking index  2014/01/02   
2014-01-03       Z8888    TSEC+GreTai Non-Banking index  2014/01/03   
2014-01-06       Z8888    TSEC+GreTai Non-Banking index  2014/01/06   

            Market Risk Premium  Size Factor (3 Factor)  \
YY/MM/DD                                                  
2014-01-02               0.0145                  0.7197   
2014-01-03              -0.5858                  0.6868   
2014-01-06              -0.4534                  0.5783   

            Book to Market Factor  Risk-free Interest Rate  
YY/MM/DD                                                    
2014-01-02                 0.3039                      NaN  
2014-01-03                 1.0533                      NaN  
2014-01-06                 0.4498                      NaN

In [23]:
data=pd.concat([UniPresident,factors.iloc[:,1:]],axis=1).dropna()
data.tail(n=3)

Uni-President                         CoName    YY/MM/DD  \
2014-12-29         1.1134  TSEC+GreTai Non-Banking index  2014/12/29   
2014-12-30         0.1001  TSEC+GreTai Non-Banking index  2014/12/30   
2014-12-31         0.4000  TSEC+GreTai Non-Banking index  2014/12/31   

            Market Risk Premium  Size Factor (3 Factor)  \
2014-12-29               0.7341                 -0.2407   
2014-12-30              -0.1990                  0.1720   
2014-12-31               0.4774                  0.0670   

            Book to Market Factor  Risk-free Interest Rate  
2014-12-29                -0.4195                -0.684285  
2014-12-30                -0.2125                -0.684285  
2014-12-31                -0.2604                -0.684285

In [18]:
data['Uni-President'] = data['Uni-President'] - data['Risk-free Interest Rate']
data['Uni-President']

2014-01-02    2.611555
2014-01-03    1.126955
2014-01-06    2.052855
2014-01-07    1.865955
2014-01-08    1.678355
                ...   
2014-12-26    2.052855
2014-12-27    2.052855
2014-12-29    3.166255
2014-12-30    2.152955
2014-12-31    2.452855
Name: Uni-President, Length: 248, dtype: float64

In [31]:
import statsmodels.api as sm
#parif=data.iloc[:,3:6]
regThrFac=sm.OLS(data['Uni-President'],sm.add_constant(data.iloc[:,3:6]))
result=regThrFac.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Uni-President   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.284
Method:                 Least Squares   F-statistic:                     33.68
Date:                Wed, 28 Apr 2021   Prob (F-statistic):           2.99e-18
Time:                        09:12:48   Log-Likelihood:                -393.91
No. Observations:                 248   AIC:                             795.8
Df Residuals:                     244   BIC:                             809.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0.0330      0.076     -0.433      0.665      -0.183       0.117
Market Risk Premium        1.0794      0.112      9.622      0.000       0.858       1.300
Size Factor (3 Factor)    -0.0568      0.159     -0.357      0.722      -0.371       0.257
Book to Market Factor     -0.0142      0.208     -0.068      0.946      -0.424       0.396
==============================================================================
Omnibus:                       46.399   Durbin-Watson:                   2.153
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              344.846
Skew:                           0.435   Prob(JB):                     1.31e-75
Kurtosis:                       8.711   Cond. No.                         2.88
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
#利用params提取模型係數
result.params

const                    -0.032954
Market Risk Premium       1.079372
Size Factor (3 Factor)   -0.056831
Book to Market Factor    -0.014195
dtype: float64